In [1]:
import torch_geometric as tg
import torch 
from utils.data import build_data, get_target, get_neighbors
from utils.training import get_model, train
import ase.io
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('../database.json')
df.dropna(inplace=True)

structures = []
for index,row in df.iterrows():
    structures.append(ase.io.read(f'../structures/{index}.cif'))
df['structure'] = structures

In [3]:
df.head()

comp                                           Freq_meV  \
0       Cs  [0.2500020894, 0.3500026531, 0.450004577300000...   
1  AuInPdY  [0.2500020894, 0.3500026531, 0.450003216700000...   
2   Pd2SnY  [0.2500020894, 0.3500026531, 0.450003216700000...   
3    Hg3Tl  [0.2500020894, 0.3500026531, 0.450004577300000...   
4   AuLiSb  [0.2500020894, 0.3500026531, 0.450003216700000...   

                                                 a2F  \
0  [0.000393028, 0.00107847, 0.002292140000000000...   
1  [0.0030494, 0.00836756, 0.0177841, 0.032470000...   
2  [0.00044628200000000005, 0.0012246, 0.00260272...   
3  [6.38383e-05, 0.00017517200000000002, 0.000372...   
4  [8.20164e-05, 0.00022505300000000002, 0.000478...   

                                          PhFreq_meV  \
0  [-6.09e-08, 0.1000008057, 0.20000167230000002,...   
1  [-2.699e-07, 0.1000005967, 0.20000146330000002...   
2  [-1.871e-07, 0.10000067950000001, 0.200001546,...   
3  [-1.1850000000000001e-07, 0.100000748, 0.20000...   
4  [-2.591e-07, 0.1000006074, 0.200001474, 0.3000...   

                                           Tot_PhDOS  \
0  [0.0, 0.0020322212, 0.008128885300000001, 0.01...   
1  [0.0, 0.0008684808, 0.0034739235, 0.0078163281...   
2  [0.0, 0.00029649550000000003, 0.0011859828, 0....   
3  [0.0, 0.000925337, 0.0037013502, 0.00832803980...   
4  [0.0, 0.000253877, 0.0010155097, 0.0022848982,...   

                                     Site_Proj_PhDOS    lambda     w_log  \
0  [[0.0, 0.0020321935, 0.0081292579, 0.018290225...  0.200493  2.004419   
1  [[0.0, 0.0001485995, 0.0005797672, 0.001271532...  2.387674  2.187430   
2  [[0.0, 6.17433e-05, 0.00024316800000000001, 0....  2.142195  3.377658   
3  [[0.0, 0.00023463460000000002, 0.0009387482000...  0.875469  3.744005   
4  [[0.0, 5.4035e-06, 2.15874e-05, 4.85118e-05, 8...  1.159056  3.959086   

         w_sq               comp_name  ...  \
0    6.257714                 Cs_mp_1  ...   
1   24.628668  Y1In1Pd1Au1_mp_1215995  ...   
2   33.592206        Y1Sn1Pd2_mp_5538  ...   
3   21.080808         Tl1Hg3_mp_11474  ...   
4  121.864371      Li1Sb1Au1_mp_12564  ...   

                                     Tot_PhDOS_dense  \
0  [0.0, 0.001248159, 0.0047586053000000005, 0.00...   
1  [0.0, 3.6431000000000004e-05, 0.0001457241, 0....   
2  [0.0, 4.44231e-05, 0.00017769270000000001, 0.0...   
3  [0.0, 0.0009582428, 0.0038329729000000003, 0.0...   
4  [0.0, 8.94992e-05, 0.0003579977, 0.0008054955,...   

                               Site_Proj_PhDOS_dense  \
0  [[0.0, 0.0012481423000000001, 0.004758588, 0.0...   
1  [[0.0, 6.3832e-06, 2.55178e-05, 5.73815e-05, 0...   
2  [[0.0, 9.386700000000001e-06, 3.75177e-05, 8.4...   
3  [[0.0, 0.00024280500000000002, 0.0009706878000...   
4  [[0.0, 1.9065e-06, 7.6235e-06, 1.71465e-05, 3....   

                                    Tot_eDOS_eng_meV  \
0  [-59.9, -58.9, -57.9, -56.9, -55.9, -54.9, -53...   
1  [-60.0, -59.0, -58.0, -57.0, -56.0, -55.0, -54...   
2  [-59.0, -58.0, -57.0, -56.0, -55.0, -54.0, -53...   
3  [-59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53...   
4  [-59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53...   

                                            Tot_eDOS  \
0  [1.2210500000000001e-05, 1.2210500000000001e-0...   
1  [2.34581e-05, 2.34581e-05, 2.34581e-05, 2.3458...   
2  [2.4449000000000003e-05, 2.4449000000000003e-0...   
3  [1.0029e-05, 1.0128300000000001e-05, 1.0128300...   
4  [2.06008e-05, 2.06008e-05, 2.06008e-05, 2.0600...   

                              Site_proj_eDOS_eng_meV  \
0  [-59.9, -58.9, -57.9, -56.9, -55.9, -54.9, -53...   
1  [-60.0, -59.0, -58.0, -57.0, -56.0, -55.0, -54...   
2  [-59.0, -58.0, -57.0, -56.0, -55.0, -54.0, -53...   
3  [-59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53...   
4  [-59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53...   

                                      Site_proj_eDOS  \
0  [[1.11784e-05, 1.12709e-05, 1.12709e-05, 1.127...   
1  [[9.0432e-06, 9.0431e-06, 9.054500000000001e-0...   
2  [[6.7767e-06, 

In [4]:
r_max = 4
df['target'] = df.apply(get_target,axis=1)
df['formula'] = df['structure'].map(lambda x: x.get_chemical_formula())
df['species'] = df['structure'].map(lambda x: list(set(x.get_chemical_symbols())))
species = sorted(list(set(df['species'].sum())))

df['data'] = df.apply(build_data,embed_ph_dos=True,embed_e_dos=False,fine=True,r_max = r_max,axis=1)


In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

out_dim = len(df.iloc[0]['target'])
in_dim = len(df.iloc[0].data.x[0])
em_dim = 64

In [6]:
init_dict_base = dict(in_dim=118+51,                           
    em_dim=em_dim,                        
    irreps_in=str(em_dim)+"x0e",          
    irreps_out=str(out_dim)+"x0e",        
    irreps_node_attr=str(em_dim)+"x0e",   
    layers=2,                             
    mul=32,                               
    lmax=1,                               
    max_radius=r_max,      
    num_neighbors=get_neighbors(df,df.index).mean(),      
    reduce_output=True,
    p=0.0
)

In [7]:
max_iter = 10

In [8]:
loss_fn = torch.nn.MSELoss()
loss_fn_mae = torch.nn.MSELoss()

In [9]:
folds = range(2)
for i in folds:
    idx_train = pd.Index(np.loadtxt(f'../indices/idx_train_V2_{i}.txt'))
    idx_valid = pd.Index(np.loadtxt(f'../indices/idx_valid_V2_{i}.txt'))
    
    dataloader_train = tg.loader.DataLoader(df.iloc[df.index.isin(idx_train)]['data'].values, batch_size=652)
    dataloader_valid = tg.loader.DataLoader(df.iloc[df.index.isin(idx_valid)]['data'].values, batch_size=164)
    name = f'../models/FPD_test_{i}'
    
    run_name = f'{name}'
    print(name)

    model, opt, scheduler = get_model(init_dict_base,lr = 0.005,wd=0,device=device)

    train(model, opt, dataloader_train, dataloader_valid,
          loss_fn, loss_fn_mae, run_name,
          max_iter=max_iter, scheduler=scheduler, device=device)

../models/FPD_test_0
Validation loss decreased (inf --> 0.010311).  Saving model ...
Validation loss decreased (0.010311 --> 0.010311).  Saving model ...
Validation loss decreased (0.010311 --> 0.010310).  Saving model ...
Validation loss decreased (0.010310 --> 0.010309).  Saving model ...
Validation loss decreased (0.010309 --> 0.010308).  Saving model ...
Validation loss decreased (0.010308 --> 0.010306).  Saving model ...
Validation loss decreased (0.010306 --> 0.010303).  Saving model ...
Validation loss decreased (0.010303 --> 0.010297).  Saving model ...
Validation loss decreased (0.010297 --> 0.010288).  Saving model ...
Validation loss decreased (0.010288 --> 0.010273).  Saving model ...
Validation loss decreased (0.010273 --> 0.010248).  Saving model ...
../models/FPD_test_1
Validation loss decreased (inf --> 0.009475).  Saving model ...
Validation loss decreased (0.009475 --> 0.009475).  Saving model ...
Validation loss decreased (0.009475 --> 0.009475).  Saving model ...
Va